# AI Assisted Annotation 

Manual annotation is slow, tedious and costly. Faster labeling of 3D volumes using AI annotation models accelerates this process. Clara Train offers an AIAA API that easily integrates into common medical imaging viewers
<br>![side_bar](screenShots/AIAASpeedup.png)

AIAA is based on a server client model as shown below 
<br>![side_bar](screenShots/AIAAClientServer.png)



By the end of this notebook you will be able to:
- Start AIAA server 
- Load a deep grow model
- Annotate using deep grow 
- Load your model and use it for annotations  
- Stop AIAA server


## Prerequisites
- Nvidia GPU with 8Gb of memory   


### Resources
It might be helpful to watch the GTC Digital 2020 talk on Clara Train SDK 
- [S22563](https://developer.nvidia.com/gtc/2020/video/S22563)
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


# Lets get started
Before we get started let us check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi



### Start AIAA server
The cell below will start the AIAA server


In [ ]:
# setting up MMAR root path
MMAR_ROOT="/claraDevDay/MMARs/GettingStarted/"
print ("setting MMAR_ROOT=",MMAR_ROOT)
AIAA_ROOT="/claraDevDay/tmp/AIAA/"
print ("AIAA_ROOT is set to ",AIAA_ROOT)


In [ ]:
# To run nfrom a terminal you should just run
# start_aas.sh --workspace <workspace> &  
import subprocess
a = subprocess.Popen(["start_aas.sh","--workspace",AIAA_ROOT])


Let us now check that the server has started by going to the url [http://localhost:5000/](http://clarasa-station:5000/) <br>
You can also check the logs at [http://localhost:5000/logs](http://clarasa-station:5000/logs) 
We could also try running `!curl -X GET "http://127.0.0.1/logs/" -H "accept: application/json"` in the terminal. 

The cell below will get the last 15 lines of the logs.

In [ ]:
!curl -X GET "http://127.0.0.1/logs/?lines=15" -H "accept: application/json" 


#### Load models in AIAA Server. 
For this notebook we can download models from [NGC](). Models on NGC are either:
- Annotation Models <br>
[Deep Extreme Cut: From Extreme Points to Object Segmentation](https://arxiv.org/abs/1711.09081)
<br>![side_bar](screenShots/AIAAAnnotation.png)
- Segmentation Models
- DeepGrow MMAR 
This is an interactive model to get you started with annotation. CNN takes in single channel (image) + use single click 
for foreground or background location then produces the segmentation. it is based on [Interactive segmentation of medical images through
fully convolution neural networks](https://arxiv.org/pdf/1903.08205.pdf)
<br>![side_bar](screenShots/AIAADeepGrow.png)



You can see a [list of available pre-trained models](https://ngc.nvidia.com/containers/nvidia:clara-train-sdk) on NGC. 
You can also use `ngc registry model list nvidia/med/clara_*` to get a list of models.

In [ ]:
!ngc registry model list nvidia/clara_*
!ngc registry model list nvidia/med/clara_*


The cell below will download the the deep grow model from NGC


In [ ]:
# model name from NGC is clara_train_deepgrow_aiaa_inference_only
!curl -X PUT "http://127.0.0.1/admin/model/clara_deepgrow" \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_train_deepgrow_aiaa_inference_only","version":"1"}'


The following cell will download the spleen model and load it into the AIAA server

In [ ]:

!curl -X PUT "http://127.0.0.1/admin/model/clara_ct_seg_spleen_no_amp" \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_ct_seg_spleen_no_amp","version":"1"}'

If you manually downloaded the MMAR you should run the command below to load the model 
```
curl -X PUT "http://127.0.0.1/admin/model/clara_ct_seg_spleen_amp" \
     -F "config=@clara_ct_seg_spleen_amp_v1/config/config_aiaa.json;type=application/json" \
     -F "data=@clara_ct_seg_spleen_amp_v1/models/model.trt.pb"
```

Also if you have the MMAR structure zipped you could do 
```
dataArg="data=@"+AIAA_ROOT+"/deepgrow_nifti.zip"
!curl -X PUT "http://127.0.0.1/admin/model/clara_deepgrow_v2" -F $dataArg
```


Let us check the server and check that the model is uploaded. after running the cell below you should see the deep grow model name returned with the description 

In [ ]:
!curl -X GET "http://127.0.0.1:80/v1/models" -H "accept: application/json"


### AIAA Clients 

AIAA server can connect to any client that implements the APIs found [here](https://github.com/NVIDIA/ai-assisted-annotation-client
). 
NVIDIA has already implemented these APIs for a number of open source viewers:
1. 3DSlicer. 
    1. Steps to install and setup slicer 3d [here](https://github.com/NVIDIA/ai-assisted-annotation-client/tree/master/slicer-plugin)
    2. Install by searching for nvidia in the plugin manager
    3. Configure plugin. Add the AIAA server location and make sure the session is enabled
       <br>![side_bar](screenShots/SlicerConfig.png)
    4. You should load a volume and start trying the spleen and deep grow model as shown below
      
<br>![side_bar](screenShots/Slicer.png)

2. MITK. You can download and install it [here](http://mitk.org/wiki/Downloads). 
Please make sure you install the release with nvidia AIAA.<br>
_Note_: Deep grow is not enabled yet in MITK 

### Stop AIAA server  

In [ ]:
!stop_aas.sh